### User Interface Code

In [4]:
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import filedialog 

# Create Window to populate
window=Tk()
window.geometry("362x155")
window.title("Pansharpening Tool")

# Options Window Label
Options_Label = ttk.Button(window, text='Options')
Options_Label.focus()
Options_Label.grid(column=2, row=7)

# Resampling Technique Selection
clicked = StringVar()
Resample_options = OptionMenu(window,clicked,"Nearest Neighbor", "Bilinear Interpolation", "Cubic Convolution","Cubicspline", "Lanczos", "Average")
Resample_options.grid(row=7,column=1)
ttk.Label(window, text='Resampling Technique:').grid(column=0, row=7, sticky=tk.W)

ttk.Label(window, text='High Resolution Input').grid(column=0, row=1, sticky=tk.W)
highres = ttk.Entry(window, width=24)
highres.focus()
highres.grid(column=1, row=1, sticky=tk.W)

# Find file location
def browseFiles():
    filename = filedialog.askopenfilename(initialdir = "/", title = "Select a File", filetypes = (("Text files","*.txt*"),("all files","*.*")))
    # Change label contents
    label_file_explorer.configure(text="File Opened: "+filename)

highres_find = ttk.Button(window, text='Find...', command= browseFiles)
highres_find.focus()
highres_find.grid(column=2, row=1)

# Multispectral File
ttk.Label(window, text='Multispectral Input').grid(column=0, row=2, sticky=tk.W)
multi = ttk.Entry(window, width=24)
multi.focus()
multi.grid(column=1, row=2, sticky=tk.W)
multi_find = ttk.Button(window, text='Find...', command= browseFiles)
multi_find.focus()
multi_find.grid(column=2, row=2)

# Output File:
ttk.Label(window, text='Output File').grid(column=0, row=8, sticky=tk.W)
output = ttk.Entry(window, width=24)
output.grid(column=1, row=8, sticky=tk.W)
output_find = ttk.Button(window, text='Find...', command= browseFiles)
output_find.focus()
output_find.grid(column=2, row=8)

# Generate Pyramids checkbox
gen_pyramids = tk.StringVar()
gen_pyramids_check = ttk.Checkbutton(
    window,
    text='Generate Pyramids',
    variable=gen_pyramids,
    command=lambda: print(gen_pyramids.get()))
gen_pyramids_check.grid(column=0, row=3, sticky=tk.W)

# Calculate Statistics checkbox
calc_statistics = tk.StringVar()
calc_statistics_check = ttk.Checkbutton(
    window,
    variable=calc_statistics,
    text='Calculate Statistics',
    command=lambda: print(calc_statistics.get()))
calc_statistics_check.grid(column=0, row=4, sticky=tk.W)

# Text input for filename
def click():
    res ="Output Filename: "+txt.get()
    Output_Label.configure(text=res)


window.mainloop()

In [ ]:
def pansharpen_simple_mean(m, pan, psh):
    """
    Inputs:
    - m: File path of multispectral image to undergo pansharpening
    - pan: File path of panchromatic image to be used for pansharpening
    - psh: File path of pansharpened multispectral image to be written to file

    """

    with rasterio.open(m) as f:
        metadata_ms = f.profile
        img_ms = np.transpose(
            f.read(tuple(np.arange(metadata_ms["count"]) + 1)), [1, 2, 0]
        )

    with rasterio.open(pan) as g:
        metadata_pan = g.profile
        img_pan = g.read(1)

    ms_to_pan_ratio = metadata_ms["transform"][0] / metadata_pan["transform"][0]
    rescaled_ms = cv2.resize(
        img_ms,
        dsize=None,
        fx=ms_to_pan_ratio,
        fy=ms_to_pan_ratio,
        interpolation=cv2.INTER_CUBIC,
    ).astype(metadata_ms["dtype"])

    if img_pan.shape[0] < rescaled_ms.shape[0]:
        ms_row_bigger = True
        rescaled_ms = rescaled_ms[: img_pan.shape[0], :, :]
    else:
        ms_row_bigger = False
        img_pan = img_pan[: rescaled_ms.shape[0], :]

    if img_pan.shape[1] < rescaled_ms.shape[1]:
        ms_column_bigger = True
        rescaled_ms = rescaled_ms[:, : img_pan.shape[1], :]
    else:
        ms_column_bigger = False
        img_pan = img_pan[:, : rescaled_ms.shape[1]]

    del img_ms
    gc.collect()

    if ms_row_bigger == True and ms_column_bigger == True:
        img_psh = np.zeros(
            (img_pan.shape[0], img_pan.shape[1], rescaled_ms.shape[2]),
            dtype=metadata_pan["dtype"],
        )
    elif ms_row_bigger == False and ms_column_bigger == True:
        img_psh = np.zeros(
            (rescaled_ms.shape[0], img_pan.shape[1], rescaled_ms.shape[2]),
            dtype=metadata_pan["dtype"],
        )
        metadata_pan["height"] = rescaled_ms.shape[0]
    elif ms_row_bigger == True and ms_column_bigger == False:
        img_psh = np.zeros(
            (img_pan.shape[0], rescaled_ms.shape[1], rescaled_ms.shape[2]),
            dtype=metadata_pan["dtype"],
        )
        metadata_pan["width"] = rescaled_ms.shape[1]
    else:
        img_psh = np.zeros((rescaled_ms.shape), dtype=metadata_pan["dtype"])
        metadata_pan["height"] = rescaled_ms.shape[0]
        metadata_pan["width"] = rescaled_ms.shape[1]

    # This is the core of the function where the simple mean is applied
    for band in range(rescaled_ms.shape[2]):
        img_psh[:, :, band] = 0.5 * (rescaled_ms[:, :, band] + img_pan)

    del img_pan, rescaled_ms
    gc.collect()

    metadata_pan["count"] = img_psh.shape[2]
    with rasterio.open(psh, "w", **metadata_pan) as dst:
        dst.write(np.transpose(img_psh, [2, 0, 1]))

    return img_psh

### Code Sources

https://docs.python.org/3/library/tkinter.html

https://www.youtube.com/watch?v=ls3BAhPV06M&ab_channel=ProgrammingKnowledge

https://www.youtube.com/watch?v=LsSimyks8jE&ab_channel=SmartProgrammer

https://www.pythontutorial.net/tkinter/tkinter-object-oriented-frame/ 

https://www.geeksforgeeks.org/file-explorer-in-python-using-tkinter/ 